# HDS5210-2022 Midterm

In the midterm, you're going to use all the programming and data management skills you've developed so far to build a risk calculator that pretends to be integrated with a clinical registry.  You'll compute the PRIEST COVID-19 Clinical Severity Score for a series of patients and, based on their risk of an adverse outcome, query a REST web service to find a hospital to transfer them to. The end result of your work will be a list of instructions on where each patient should be discharged given their risk and various characteristics of the patient.

Each step in the midterm will build up to form your complete solution.

**Make sure you write good docstrings and doctests along the way!!**

**The midterm is due at 11:59 PM CST on Monday, October 23rd.**

---

## Step 1: Calculate PRIEST Clinical Severity Score

This scoring algorithm can be found [here on the MDCalc website](https://www.mdcalc.com/priest-covid-19-clinical-severity-score#evidence).  

1. You will need to write a function called **priest()** with the following input parameters.  
 * Sex (Gender assigned at birth)
 * Age in years
 * Respiratory rate in breaths per minute
 * Oxygen saturation as a percent between 0 and 1
 * Heart rate in beats per minute
 * Systolic BP in mmHg
 * Temperature in degrees C
 * Alertness as a string description
 * Inspired Oxygen as as string description
 * Performance Status as a string description
2. The function will need to follow the algorithm provided on the MDCalc website to compute a risk percentage that should be returned as a numeric value between 0 and 1.
3. Be sure to use docstring documentation and at least three built-in docstring test cases.
4. Assume that the input values that are strings could be any combination of upper or lower case. For example: 'male', 'Male', 'MALE', 'MalE' should all be interpretted by your code as male.
5. If any of the inputs are invalid (for example a sex value that is not recognizable as male or female) your code should return None.

NOTES:
1. In the final step there is a table that translates from **PRIEST Score** to **30-day probability of an outcome** but the last two probabilities are shown as ranges (59-88% and >99%).  Our code needs to output a single number, however. For our code, use the following rule:
 * If PRIEST score is between 17 and 25, the probability you return should be 0.59
 * If PRIEST score is greater than or equal to 26, the probability you return should be 0.99


In [40]:
def priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP, temperature, alertness, inspired_oxygen, performance_status):
    """
    (str, int, int, float, int, int, float, str, str, str) -> (float)
    The priest function initially determines an individual values of COVID-19 severity based on the above given factors.
    The algorithm used is available on the MDCalc website.
    Based on the severity, we may then predict the probability that the patient would have any adverse effects within the next 30 days.

    >>> priest("female", 58, 18, 0.94, 88, 112, 38.3, 'Alert', 'air', 'Unrestricted normal activity')
    0.03
    >>> priest ("FemalE", 78, 15, 0.92, 100, 200, 31.0, 'confused or not alert', 'supplemental OxYGEN', 'limited strenuous activity, can do light activity')
    0.49
    >>> priest("MaLe", 85, 19, 0.90, 80, 115, 38.2, 'CONFUSED OR not ALERT','SUPPLEMENTAL OxygEN','LIMITED activity, can SELF-CARE')
    0.55
    >>> priest('FEMALE', 66, 19, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care')
    0.34
    >>> priest('female', 51, 23.0, 0.94, 135, 115, 38.1, 'ALERT', 'AIR', 'limited strenuous activity, can do light activity')
    0.29
    """

    score = 0
    sex = sex.lower()
    if sex == "male":
        score += 1
    elif sex != "female":
        return None

    if 16 <= age <= 49:
        score += 0
    elif 50 <= age <= 65:
        score += 2
    elif 66 <= age <= 80:
        score += 3
    elif age > 80:
        score += 4

    if respiratory_rate < 9:
        score += 3
    elif 9 <= respiratory_rate <= 11:
        score += 1
    elif 12 <= respiratory_rate <= 20:
        score += 0
    elif 21 <= respiratory_rate <= 24:
        score += 2
    elif respiratory_rate > 24:
        score += 3

    if oxygen_saturation > 0.95:
        score += 0
    elif 0.94 <= oxygen_saturation <= 0.95:
        score += 1
    elif oxygen_saturation >= 0.92 <= 0.93:
        score += 2
    elif oxygen_saturation < 0.92:
        score += 3

    if heart_rate < 41:
        score += 3
    elif 41 <= heart_rate <= 50:
        score += 1
    elif 51 <= heart_rate <= 90:
        score += 0
    elif 91 <= heart_rate <= 110:
        score += 1
    elif 111 <= heart_rate <= 130:
        score += 2
    elif heart_rate > 130:
        score += 3

    if systolic_BP < 91:
        score += 3
    elif 91 <= systolic_BP <= 100:
        score += 2
    elif 101 <= systolic_BP <= 110:
        score += 1
    elif 111 <= systolic_BP <= 219:
        score += 0
    elif systolic_BP > 219:
        score += 3

    if temperature < 35.1:
        score += 3
    elif 35.1 <= temperature <= 36.0:
        score += 1
    elif 36.1 <= temperature <= 38.0:
        score += 0
    elif 38.1 <= temperature <= 39.0:
        score += 1
    elif temperature > 39.0:
        score += 2

    alertness = alertness.lower()
    if alertness == "alert":
        score += 0
    elif alertness == "confused or not alert":
        score += 3

    if inspired_oxygen.lower() == "air":
        score += 0
    elif inspired_oxygen.lower() == "supplemental oxygen":
        score += 2

    if performance_status.lower() == "unrestricted normal activity":
        score += 0
    elif performance_status.lower() == "limited strenuous activity, can do light activity":
        score += 1
    elif performance_status.lower() == "limited activity, can self-care":
        score += 2
    elif performance_status.lower() == "limited self-care":
        score += 3
    elif performance_status.lower() == "bed/chair bound, no self-care":
        score += 4

    risk = 0.0
    if 0 <= score <= 1:
        risk = 0.01
    elif 2 <= score <= 3:
        risk = 0.02
    elif score == 4:
        risk = 0.03
    elif score == 5:
        risk = 0.09
    elif score == 6:
        risk = 0.15
    elif score == 7:
        risk = 0.18
    elif score == 8:
        risk = 0.22
    elif score == 9:
        risk = 0.26
    elif score == 10:
        risk = 0.29
    elif score == 11:
        risk = 0.34
    elif score == 12:
        risk = 0.38
    elif score == 13:
        risk = 0.46
    elif score == 14:
        risk = 0.47
    elif score == 15:
        risk = 0.49
    elif score == 16:
        risk = 0.55
    elif 17 <= score <= 25:
        risk = 0.59
    elif score >= 26:
        risk = 0.99

    return risk

In [41]:
import doctest
doctest.run_docstring_examples(priest, globals(),verbose=True)

Finding tests in NoName
Trying:
    priest("female", 58, 18, 0.94, 88, 112, 38.3, 'Alert', 'air', 'Unrestricted normal activity')
Expecting:
    0.03
ok
Trying:
    priest ("FemalE", 78, 15, 0.92, 100, 200, 31.0, 'confused or not alert', 'supplemental OxYGEN', 'limited strenuous activity, can do light activity')
Expecting:
    0.49
ok
Trying:
    priest("MaLe", 85, 19, 0.90, 80, 115, 38.2, 'CONFUSED OR not ALERT','SUPPLEMENTAL OxygEN','LIMITED activity, can SELF-CARE')
Expecting:
    0.55
ok
Trying:
    priest('FEMALE', 66, 19, 0.96, 135, 115, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care')
Expecting:
    0.34
ok
Trying:
    priest('female', 51, 23.0, 0.94, 135, 115, 38.1, 'ALERT', 'AIR', 'limited strenuous activity, can do light activity')
Expecting:
    0.29
ok


## Part 2: Find a hospital

The next thing we have to do is figure out where to send this particular patient.  The guidelines on where to send a patient are based on their age (pediatric, adult, geriatric), sex, and risk percentage.  Luckily, you don't have to implement these rules. I already have. All you have to do is use a REST web service that I've created for you.

You'll want to use Python to make a call to my REST web service similar to the example URL below. The first part of the URL will be the same for everyone and every request that you make. What you will need to modify for each of your requests is the information after the question mark.

```
https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age=40&sex=male&risk_pct=0.1
```

The example above asks my web service where a 40-year old male with a risk of 10% should go.  What the web service will return back is a JSON string containing the information you need.  That JSON will look like this:

```json
{
  "age": "40",
  "sex": "male",
  "risk": "0.1",
  "hospital": "Southwest Hospital and Medical Center"
}
```

My function is not smart enough to understand `'MALE'` is the same as `'male'`.  You have to send it exactly `'male'` or `'female'`

1. Your job is to write a function called **find_hospital()** that takes age, sex, and risk as parameters.
2. Your function should call this REST web service using the `requests` module
3. Then your function will need to interpret the JSON it gets and return just the name of the hospital
4. If anything fails, return None
5. Include a good docstring with at least three test cases.


In [42]:
import requests
import doctest

def find_hospital(age, sex, risk):
    """Returns the hospital name give the age, sex, and risk factor.
    Invalid value for age returns None:
    >>> find_hospital(-1, "female", 0.4)

    Invalid value for sex returns None:
    >>> find_hospital(30, "xxxx", 0.5)

    Invalid value for risk returns None:
    >>> find_hospital(42, "male", 1.1)

    >>> find_hospital(40, "male", 0.1)
    'Southwest Hospital and Medical Center'

    >>> find_hospital(35, "female", 0.7)
    'Emory Dunwoody Medical Center'
    """

    api_url = "https://oumdj6oci2.execute-api.us-east-1.amazonaws.com/prd/?age={0}&sex={1}&risk_pct={2}"

    if sex.lower() not in ["male", "female"]:
        return None
    if age < 0:
        return None
    if risk < 0.0 or risk > 1.0:
        return None
    response = requests.get(api_url.format(age, sex, risk))
    if response.status_code == 200:
        return response.json()["hospital"]
    else:
        return None

In [43]:
import doctest
doctest.run_docstring_examples(find_hospital, globals(),verbose=True)

Finding tests in NoName
Trying:
    find_hospital(-1, "female", 0.4)
Expecting nothing
ok
Trying:
    find_hospital(30, "xxxx", 0.5)
Expecting nothing
ok
Trying:
    find_hospital(42, "male", 1.1)
Expecting nothing
ok
Trying:
    find_hospital(40, "male", 0.1)
Expecting:
    'Southwest Hospital and Medical Center'
ok
Trying:
    find_hospital(35, "female", 0.7)
Expecting:
    'Emory Dunwoody Medical Center'
ok


## Part 3: Get the address for that hospital from a JSON file

Great! Now we have code to tell us which hospital to send someone to... but we don't know where that hospital is. The next function we need to create is one that looks up the address of that hospital.  All of these hospitals are in Atlanta, Georgia.  We're going to use the list from this webpage to lookup the address for that hospital, based on its name.  https://www.officialusa.com/stateguides/health/hospitals/georgia.html

Because we skipped the section about Beautiful Soup and working with HTML, I've converted this information into a JSON document for you.  It's available for you here.  Your code should retrieve this file using the `requests` module.

`https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-`

1. You need to create a function called **get_address()** that takes hospital name as a parameter and searches the data from this JSON file for the hospital you want to find.
2. Your code will have to load the JSON and return the correct hospital based on name.
3. If the hospital name isn't found, the function should return None.
4. Be sure to use good docstring documentation and includes at least 3 test cases.

In [44]:
import requests
import doctest

def get_address_from_url(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None

api_url = "https://drive.google.com/uc?export=download&id=1fIFD-NkcdiMu941N4GjyMDWxiKsFJBw-"
hospital_addresses = get_address_from_url(api_url)

def get_address(hospital_name):
    """
    :param hospital_name:
    :return: hospital_address
    >>> get_address("")
    >>> get_address("unknown hospital")
    >>> get_address("Emory Dunwoody Medical Center")
    '4500 NORTH SHALLOWFORD ROAD'
    """
    hospital_name = hospital_name.upper()
    if hospital_name in hospital_addresses:
        return hospital_addresses[hospital_name]["ADDRESS"]
    else:
        return None

In [45]:
import doctest
doctest.run_docstring_examples(get_address, globals(), verbose=True)

Finding tests in NoName
Trying:
    get_address("")
Expecting nothing
ok
Trying:
    get_address("unknown hospital")
Expecting nothing
ok
Trying:
    get_address("Emory Dunwoody Medical Center")
Expecting:
    '4500 NORTH SHALLOWFORD ROAD'
ok


## Part 4: Run the risk calculator on a population

At the link below, there is a file called `people.psv`.  It is a pipe-delimited (`|`) file with columns that match the inputs for the PRIEST calculation above.  Your code should use the `requests` module to retrieve the file from this URL.

`https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh`


In addition, the file has a patient identifier in the first column.

1. Write a function called **process_people()** that takes the file location above as its only parameter. Your Python program should use your code above to process all of these rows, determine the hospital and address, and return a list whose items are a dictionary like this: `{ patient_number: [sex, age, breath, o2sat, heart, systolic, temp, alertness, inspired, status, hospital, address]}`.  Look at the file in Part 5 for what the output looks like.
2. Be sure to use good docstrings, but you don't need any tests in your doc strings.  I've provided those for you withe file in Part 5.


**NOTE** that when running your code for all the 100 records in the `people.psv` file, it may take a few minutes to complete.  You're making multiple calls to the internet for each record, so that can take a little while.


In [46]:
import requests

def process_people(url):
    """
    This function reads the csv file from the given URL and prepares a dictionary with patient details.
    :url: The Google Drive URL
    :return: the dictionary with patient details.
    """
    people = {}
    response = requests.get(url)
    for row in response.text.split("\n")[1:]:
        fields = row.strip().split("|")
        if(len(fields) == 11):
            patient_id = fields[0]
            (sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
            temperature, alertness, inspired_oxygen, performance_status) = (fields[1], int(fields[2]), float(fields[3]), float(fields[4]),
                float(fields[5]), float(fields[6]), float(fields[7]), fields[8], fields[9], fields[10])
            risk = priest(sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
                          temperature, alertness, inspired_oxygen, performance_status)
            hospital = find_hospital(age, sex.lower(), risk)
            address = get_address(hospital)
            people[patient_id] = [sex, age, respiratory_rate, oxygen_saturation, heart_rate, systolic_BP,
                                  temperature, alertness, inspired_oxygen, performance_status, risk, hospital, address]
    return people

In [47]:
import doctest
print(people)

{'E9559': ['FEMALE', 40, 24.0, 0.96, 105.0, 115.0, 34.9, 'ALERT', 'AIR', 'unrestricted normal activity', 0.15, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9385': ['Female', 51, 19.0, 0.99, 90.0, 91.0, 38.1, 'ALERT', 'supplemental oxygen', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E3067': ['female', 40, 29.0, 0.96, 105.0, 95.0, 38.1, 'ALERT', 'air', 'unrestricted normal activity', 0.18, 'Select Specialty Hospital - Northeast Atlanta', '1821 CLIFTON ROAD NE'], 'E9422': ['FEMALE', 66, 19.0, 0.96, 135.0, 115.0, 38.1, 'ALERT', 'AIR', 'bed/chair bound, no self-care', 0.34, 'Wesley Woods Geriatric Hospital', '1821 CLIFTON ROAD, NE'], 'E8661': ['MALE', 18, 22.0, 0.95, 105.0, 115.0, 36.3, 'Confused or Not Alert', 'air', 'unrestricted normal activity', 0.22, 'Southwest Hospital and Medical Center', '501 FAIRBURN ROAD SW'], 'E6235': ['Male', 16, 23.0, 0.96, 135.0, 115.0, 39.2, 'ALERT', 'suppleme

## Part 5: Checking your final results

The final step is to check your results.  You should be able to compare your results to the output in `people_results.json` at the link below.  Write some code to check your results.  This does not need to be a function.

`https://drive.google.com/uc?export=download&id=1gx1SSC20mO5XL6uYD0mdcM_cL91fcIW5`


In [48]:
import json

def check_results(people):
    f = open('people_results.json')
    data = json.load(f)
    passed = True
    for patient_id in people.keys():
        if patient_id in data:
            if people[patient_id] != data[patient_id]:
                print(people[patient_id])
                print(data[patient_id])
                passed = False
                break
        else:
            passed = False
            print(patient_id, "mismatches")
            break

    if passed:
        print("Test Passed.")
    else:
        print("Test Failed!")

In [49]:
people = process_people("https://drive.google.com/uc?export=download&id=1fLxJN9YGUqmqExrilxSS8furwUER5HHh")
check_results(people)

Test Passed.


---

## Check your work above

If you didn't get them all correct, take a few minutes to think through those that aren't correct.


## Submitting Your Work

Submit your work as usual into a folder named `midterm`

---